# 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np

# 데이터 불러오기
data_path = 'data/'

train = pd.read_csv(data_path + 'train.csv', index_col='id') # 학습 데이터
test = pd.read_csv(data_path + 'test.csv', index_col='id') # 테스트 데이터
submission = pd.read_csv(data_path + 'sample_submission.csv') # 제출 샘플 데이터

train.shape, test.shape, submission.shape

((6255, 12), (180, 11), (180, 2))

# 피처 엔지니어링

In [2]:
# 분석6 : 피처 결측치
train = train.fillna(0)
test = test.fillna(0)

In [3]:
# 데이터 합치기
df = pd.concat([train, test])

df.shape

(6435, 12)

In [4]:
# 분석1 : 피처 변환
df['Weekly_Sales'] = np.log1p(df['Weekly_Sales'])

In [5]:
# 분석2 : 피처 추가
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day
df['week'] = df['day'] // 7

In [6]:
# 분석3, 4 : 피처 제거
drop_features = ['Date', 'Fuel_Price']
df = df.drop(columns=drop_features)

In [7]:
# 분석5 : 피처 인코딩
df['IsHoliday'] = df['IsHoliday'].astype(int)

In [8]:
# 데이터셋 나누기
features = df.loc[df['Weekly_Sales'].notnull()]
label = features.pop('Weekly_Sales')

X_test = df[df['Weekly_Sales'].isnull()].drop(columns='Weekly_Sales')

features.shape, label.shape, X_test.shape

((6255, 13), (6255,), (180, 13))

# 모델 및 하이퍼파라미터 최적화

- 규제 선형 모델
    - Ridge
    - Lasso
    - Elastic net
- 트리
    - Decision Tree
    - Gradient Boosting
    - RandomForest
    - XGBoost
    - LightGBM

In [9]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

n = 15

kfold = KFold(n_splits=n, shuffle=True, random_state=2001) # KFold

def kfold_rmse(model):
    """KFold 교차 검증으로 모델 학습 및 검증"""
    n_iter = 0
    cv_rmse = []
    pred_test = np.zeros((test.shape[0]))

    for train_index, test_index in kfold.split(features):
        n_iter += 1

        # 학습, 검증 데이터 나누기
        X_train, X_val = features.iloc[train_index], features.iloc[test_index]
        y_train, y_val = label.iloc[train_index], label.iloc[test_index]

        model.fit(X_train, y_train) # 모델 학습

        pred = model.predict(X_val) # 검증용 예측
        pred_test += model.predict(X_test) / n # 제출용 예측

        rmse = np.round(mean_squared_error(y_val, pred, squared=False), 4) # 평가

        print(f'\n#{n_iter} 교차 검증 rmse : {rmse}')
        cv_rmse.append(rmse)

    print(f'\n## 평균 검증 rmse :', np.round(np.mean(cv_rmse), 4))
    
    return pred_test

## 규제 선형 모델


- Ridge model(L2 규제)
- Lasso model(L1 규제)
- Elastic Net model(L2 + L1 규제)

알다시피 선형 모델인 Linear model과 별 차이 없기 때문에 따로 제출하지 않았다.

In [10]:
from sklearn.linear_model import Ridge

ridge_model = Ridge(random_state=2001) # 릿지 모델

ridge_pred = kfold_rmse(ridge_model) # 교차 검증 및 예측


#1 교차 검증 rmse : 0.5445

#2 교차 검증 rmse : 0.5422

#3 교차 검증 rmse : 0.5516

#4 교차 검증 rmse : 0.5586

#5 교차 검증 rmse : 0.5502

#6 교차 검증 rmse : 0.5262

#7 교차 검증 rmse : 0.5372

#8 교차 검증 rmse : 0.5255

#9 교차 검증 rmse : 0.5282

#10 교차 검증 rmse : 0.5311

#11 교차 검증 rmse : 0.528

#12 교차 검증 rmse : 0.5445

#13 교차 검증 rmse : 0.5415

#14 교차 검증 rmse : 0.5375

#15 교차 검증 rmse : 0.5565

## 평균 검증 rmse : 0.5402


In [11]:
from sklearn.linear_model import Lasso

lasso_model = Lasso(random_state=2001) # 라쏘 모델

lasso_pred = kfold_rmse(lasso_model) # 교차 검증 및 예측


#1 교차 검증 rmse : 0.5579

#2 교차 검증 rmse : 0.5612

#3 교차 검증 rmse : 0.5677

#4 교차 검증 rmse : 0.5746

#5 교차 검증 rmse : 0.5553

#6 교차 검증 rmse : 0.5397

#7 교차 검증 rmse : 0.5566

#8 교차 검증 rmse : 0.5341

#9 교차 검증 rmse : 0.5375

#10 교차 검증 rmse : 0.5504

#11 교차 검증 rmse : 0.5439

#12 교차 검증 rmse : 0.5511

#13 교차 검증 rmse : 0.5524

#14 교차 검증 rmse : 0.5414

#15 교차 검증 rmse : 0.5762

## 평균 검증 rmse : 0.5533


In [12]:
from sklearn.linear_model import ElasticNet

en_model = ElasticNet(random_state=2001) # 엘라스틱넷 모델

en_pred = kfold_rmse(en_model) # 교차 검증 및 예측


#1 교차 검증 rmse : 0.5521

#2 교차 검증 rmse : 0.5528

#3 교차 검증 rmse : 0.5608

#4 교차 검증 rmse : 0.5685

#5 교차 검증 rmse : 0.5525

#6 교차 검증 rmse : 0.5358

#7 교차 검증 rmse : 0.5512

#8 교차 검증 rmse : 0.5304

#9 교차 검증 rmse : 0.5345

#10 교차 검증 rmse : 0.5418

#11 교차 검증 rmse : 0.5374

#12 교차 검증 rmse : 0.5497

#13 교차 검증 rmse : 0.547

#14 교차 검증 rmse : 0.5377

#15 교차 검증 rmse : 0.5684

## 평균 검증 rmse : 0.548


## 트리

- Decision Tree
- Gradient Boosting
- RandomForest
- XGBoost
- LightGBM

In [13]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

dt_model = DecisionTreeRegressor(random_state=2001, max_depth=100) # 결정 트리 모델 생성
gb_model = GradientBoostingRegressor(random_state=2001, n_estimators=1000) # GradientBoosting 모델 생성
rf_model = RandomForestRegressor(random_state=2001, n_estimators=1000) # 랜덤 포레스트 모델 생성
xgb_model = XGBRegressor(random_state=2001, n_estimators=1000) # XGB 모델 생성
lgbm_model = LGBMRegressor(random_state=2001, n_estimators=1000) # LGM 모델 생성

In [14]:
# DecisionTreeRegressor
dt_pred = kfold_rmse(dt_model)


#1 교차 검증 rmse : 0.1169

#2 교차 검증 rmse : 0.1416

#3 교차 검증 rmse : 0.1291

#4 교차 검증 rmse : 0.1019

#5 교차 검증 rmse : 0.1226

#6 교차 검증 rmse : 0.1085

#7 교차 검증 rmse : 0.14

#8 교차 검증 rmse : 0.1129

#9 교차 검증 rmse : 0.0935

#10 교차 검증 rmse : 0.129

#11 교차 검증 rmse : 0.1316

#12 교차 검증 rmse : 0.1104

#13 교차 검증 rmse : 0.1508

#14 교차 검증 rmse : 0.1153

#15 교차 검증 rmse : 0.1127

## 평균 검증 rmse : 0.1211


In [15]:
# GradientBoostingRegressor
gb_pred = kfold_rmse(gb_model)


#1 교차 검증 rmse : 0.0693

#2 교차 검증 rmse : 0.0686

#3 교차 검증 rmse : 0.0638

#4 교차 검증 rmse : 0.0711

#5 교차 검증 rmse : 0.0738

#6 교차 검증 rmse : 0.071

#7 교차 검증 rmse : 0.0768

#8 교차 검증 rmse : 0.0704

#9 교차 검증 rmse : 0.0618

#10 교차 검증 rmse : 0.0708

#11 교차 검증 rmse : 0.0689

#12 교차 검증 rmse : 0.0656

#13 교차 검증 rmse : 0.0713

#14 교차 검증 rmse : 0.0682

#15 교차 검증 rmse : 0.0692

## 평균 검증 rmse : 0.0694


In [16]:
# RandomForestRegressor
rf_pred = kfold_rmse(rf_model)


#1 교차 검증 rmse : 0.1004

#2 교차 검증 rmse : 0.1122

#3 교차 검증 rmse : 0.0993

#4 교차 검증 rmse : 0.0883

#5 교차 검증 rmse : 0.098

#6 교차 검증 rmse : 0.1014

#7 교차 검증 rmse : 0.1076

#8 교차 검증 rmse : 0.0923

#9 교차 검증 rmse : 0.0797

#10 교차 검증 rmse : 0.0882

#11 교차 검증 rmse : 0.1006

#12 교차 검증 rmse : 0.0797

#13 교차 검증 rmse : 0.1054

#14 교차 검증 rmse : 0.0908

#15 교차 검증 rmse : 0.083

## 평균 검증 rmse : 0.0951


In [17]:
# XGBRegressor
xgb_pred = kfold_rmse(xgb_model)


#1 교차 검증 rmse : 0.0616

#2 교차 검증 rmse : 0.0631

#3 교차 검증 rmse : 0.0674

#4 교차 검증 rmse : 0.0771

#5 교차 검증 rmse : 0.0661

#6 교차 검증 rmse : 0.0639

#7 교차 검증 rmse : 0.0767

#8 교차 검증 rmse : 0.0635

#9 교차 검증 rmse : 0.0665

#10 교차 검증 rmse : 0.07

#11 교차 검증 rmse : 0.0669

#12 교차 검증 rmse : 0.0693

#13 교차 검증 rmse : 0.063

#14 교차 검증 rmse : 0.0717

#15 교차 검증 rmse : 0.0666

## 평균 검증 rmse : 0.0676


In [18]:
# LGBMRegressor
lgbm_pred = kfold_rmse(lgbm_model)


#1 교차 검증 rmse : 0.0615

#2 교차 검증 rmse : 0.0601

#3 교차 검증 rmse : 0.0639

#4 교차 검증 rmse : 0.0693

#5 교차 검증 rmse : 0.0641

#6 교차 검증 rmse : 0.0652

#7 교차 검증 rmse : 0.0756

#8 교차 검증 rmse : 0.0717

#9 교차 검증 rmse : 0.0644

#10 교차 검증 rmse : 0.0666

#11 교차 검증 rmse : 0.0568

#12 교차 검증 rmse : 0.0658

#13 교차 검증 rmse : 0.0568

#14 교차 검증 rmse : 0.0653

#15 교차 검증 rmse : 0.0556

## 평균 검증 rmse : 0.0642


# 제출

In [19]:
def make_submit_csv(model, pred):
    """제출 파일 만들기"""
    submission['Weekly_Sales'] = np.expm1(pred)
    submission.to_csv(data_path + f'submission_{model}.csv', index=False)

In [20]:
# 기본 모델 일괄 저장 완료
# make_submit_csv('dt', dt_pred) # DecisionTreeRegressor
# make_submit_csv('gb', gb_pred) # GradientBoostingRegressor
# make_submit_csv('rf', rf_pred) # RandomForestRegressor
# make_submit_csv('xgb', xgb_pred) # XGBRegressor
# make_submit_csv('lgbm', lgbm_pred) # LGBMRegressor